In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
#FatosGeração (ID empreendiemnto) > fatodadosmetereologicos (vento, precip, etc) > dimempreendimentos (tipogeração)

#dim_empreendimentos -> id;NomEmpreendimento;SigUFPrincipal;SigTipoGeracao;NomFonteCombustivel
#fato_dados_metereologicos -> id;Data;Hora UTC;precipTotalHorario;ventoVeloHoraria;tempAr;radGlobal;umidRelAr;id_regiao
#fato_geracao -> id;DatGeracaoConjuntoDados;MdaPotenciaFiscalizadaKw;MdaPotenciaOutorgadaKw;Id_empreendimento
#id;UF;latitude;longitude;altitude
#Pegar empreendimento (id) > Pegar todos os dados meterologicos (preciptação, vento, temp...) > Pegar qual o TipoGeração na região

dim_regiao = pd.read_csv(
    "data/dim_regiao.csv",
    delimiter=";",
    encoding="UTF-8")

dim_empreendimentos = pd.read_csv(
    "data/dim_empreendimento.csv",
    delimiter=";",
    encoding="UTF-8")

fato_dados_metereologicos = pd.read_csv(
    "data/fato_dados_meteorologicos.csv",
    delimiter=";",
    encoding="UTF-8")

fato_geracao = pd.read_csv(
    "data/fato_geracao.csv",
    delimiter=";",
    encoding="UTF-8",
    decimal=',')

#sklearn.ensemble.HistGradientBoostingClassifier
print(dim_regiao.head())
print(dim_empreendimentos.head())
print(fato_dados_metereologicos.head())
print(fato_geracao.head())

In [ ]:
# dim_empreendimentos['DscMuninicpios'].str.split('-')[0][0].strip()
# melhora o nome da coluna para Municipios e filtra os nomes dos municipios para um padrão
dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].apply(lambda x: unidecode(str(x).split('-')[0].strip().upper()))
dim_empreendimentos['Municipios'] = dim_empreendimentos['DscMuninicpios']
dim_empreendimentos.drop('DscMuninicpios', axis=1, inplace=True)

dim_empreendimentos

In [ ]:
#dim_empreendimentos['Municipios'] = dim_empreendimentos['Municipios'].str.split('-', 1).str[0]
# dim_empreendimentos['Municipios'] = dim_empreendimentos['Municipios'].str.split('-')[0][0].strip()
# dim_empreendimentos['Municipios'] = dim_empreendimentos['Municipios'].str.upper()

#Cidade, MediaPrecipTotalHorarioCidade, MediaVentoVeloHorariaCidade, MediaTempArCidade, ...., QuantidadeEnergiaEnergiaSolarGeradaPorCidade, QuantidadeEnergiaNuclearGeradaPorCidade, 
# QuantidadeEnergiaTermalGeradaPorCidade, .....


empreendimentos_combinados = pd.merge(fato_geracao, dim_empreendimentos, left_on='Id_empreendimento', right_on='id')
#dim_empreendimentos.head()

#empreendimentos_combinados = empreendimentos_combinados.drop('DatGeracaoConjuntoDados', axis=1)
#empreendimentos_combinados

empreendimentos_combinados = empreendimentos_combinados.groupby(['Municipios','SigTipoGeracao'])['MdaPotenciaOutorgadaKw'].aggregate('sum').unstack()

In [ ]:
dim_regiao.head()

In [ ]:
dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao[['id', 'UF','cidade']], left_on='id_regiao', right_on='id')

#Tratando dados decimais para virarem tipo float
#dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao, left_on='id_regiao', right_on='id')

dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(str)
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].str.replace(',', '.')
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(float)

dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(str)
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].str.replace(',', '.')
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(float)

dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(str)
dados_combinados['tempAr'] = dados_combinados['tempAr'].str.replace(',', '.')
dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(float)

dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(str)
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].str.replace(',', '.')
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(float)

dados_combinados.head()

In [ ]:
dados_combinados.drop('id_y', axis=1, inplace=True)
dados_combinados.drop('id_x', axis=1, inplace=True)

dados_combinados

In [ ]:
# agrupar por cidade
# fazer media de 'precipTotalHorario', 'ventoVeloHoraria', 'tempAr', 'radGlobal', 'umidRelAr'
colunas_medias = ['precipTotalHorario', 'ventoVeloHoraria', 'tempAr', 'radGlobal', 'umidRelAr']
dados_combinados = dados_combinados.groupby('cidade')[colunas_medias].mean().reset_index()
dados_combinados.rename(columns={
  'precipTotalHorario': 'media_preciptacao',
  'ventoVeloHoraria': 'media_vento',
  'tempAr': 'media_temperaturaAr',
  'radGlobal': 'media_radiacao',
  'umidRelAr': 'media_umidade'})
#, inplace=True
dados_combinados

In [ ]:
df_final = pd.merge(dados_combinados, empreendimentos_combinados, left_on='cidade', right_on='Municipios')
df_final[['CGH', 'EOL' , 'PCH', 'UFV', 'UHE', 'UTE', 'UTN']] = df_final[['CGH', 'EOL' , 'PCH', 'UFV', 'UHE', 'UTE', 'UTN']].fillna(0)
df_final


In [168]:
# Separar as features (dados climáticos) e o target (tipo de geração de energia)
features = df_final[['precipTotalHorario','ventoVeloHoraria','tempAr','radGlobal','umidRelAr']]  # substitua pelas colunas relevantes
target = df_final[['CGH', 'EOL' , 'PCH', 'UFV', 'UHE', 'UTE', 'UTN']]

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Criar uma instância do modelo de árvore de decisão
model = MultiOutputClassifier(DecisionTreeRegressor())

# Treinar o modelo de árvore de decisão
model.fit(X_train, np.ravel(y_train))

# Fazer previsões utilizando o conjunto de teste
y_pred = model.predict(X_test)

# Avaliar a precisão do modelo
mse = mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Erro quadrático médio (MSE):", mse)
print("Precisão do modelo:", mse)

ValueError: Unknown label type: 'continuous'

In [ ]:
colunas_com_nan = df_final.columns[df_final.isnull().any()].tolist()
print("Colunas com valores NaN:")
for coluna in colunas_com_nan:
    print(coluna)

In [ ]:
# Separar as features (dados climáticos) e os targets (tipos de geração de energia)
features = df_final[['precipTotalHorario','ventoVeloHoraria','tempAr','radGlobal','umidRelAr']]
target = df_final[['CGH', 'EOL' , 'PCH', 'UFV', 'UHE', 'UTE', 'UTN']]

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Criar uma instância do modelo de Classificador de Regressão Logística Multilabel
model = MultiOutputClassifier(LogisticRegression())

# Treinar o modelo de Classificador de Regressão Logística Multilabel
model.fit(X_train, y_train)

# Fazer previsões utilizando o conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
print(classification_report(y_test, y_pred))